In [ ]:
"""
Spectroscopic Ellipsometry Data Simulator for Machine Learning
==============================================================

Overview
--------
This script generates paired Ψ/Δ ellipsometry spectra and JSON metadata
(layer stack, thicknesses, AOI, oscillator parameters) to build comprehensive
training datasets for ML models in optical-materials analysis.

Key Features
------------
- Random multilayer stack generation:
    • Surface roughness via Bruggeman EMA  
    • Bulk films using Tauc–Lorentz & Cody–Lorentz oscillators  
    • Optional metallic films (Drude + Lorentz)  
    • Native-oxide and substrate base layers  
- Optical calculations:
    • Angle propagation via Snell’s Law  
    • Fresnel reflection/transmission coefficients  
    • Full 2×2 scattering-matrix assembly for Ψ and Δ  
- Output:
    • Spectra CSV files (Ψ, Δ vs. wavelength)  
    • JSON metadata files with timestamped filenames  

Dependencies
------------
- Python ≥ 3.7  
- numpy  
- pandas  
- scipy  

Usage
-----
1. Set OUTPUT_FOLDER at the top of this file to your desired directory.  
2. Verify base-material CSVs are present in the configured path.  
3. From the command line, run:
       python datase.py  

"""

# -----------------------------------------------------------------------------
# === Standard Library Imports ===
import os                              # File and directory operations
import json                            # Read/write JSON metadata
import random                          # Random sampling for simulation parameters
import math                            # Basic real-valued math functions
import cmath                           # Complex-number math (where needed)
import time                            # Time-related utilities (e.g., sleep, timestamps)
from datetime import datetime         # Convenient datetime class for timestamping

# === Other Library Imports ===
import numpy as np                     # Array operations and numerical routines
import pandas as pd                    # Tabular data structures (DataFrames)
from scipy.optimize import minimize    # Optimization routines for fitting
from scipy.interpolate import interp1d # 1D interpolation for matching experimental data

# -----------------------------------------------------------------------------
def fresnel_coefficients(N, angles):
    """
    Calculate Fresnel reflection (r) and transmission (t) coefficients
    for s- and p-polarized light at each interface.

    Parameters:
    - N: array, shape (L, P)
         Complex refractive indices for L layers over P wavelengths.
    - angles: array, shape (L, P)
         Angles inside each layer (radians).

    Returns:
    - rs, rp: reflection coefficients for s- and p-polarization (shape (L-1, P))
    - ts, tp: transmission coefficients for s- and p-polarization (shape (L-1, P))
    """
    # 1) Identify media on either side of each interface
    n1, n2 = N[:-1], N[1:]            # before vs. after interface
    t1, t2 = angles[:-1], angles[1:]  # corresponding angles

    # 2) Compute cosines of those angles
    c1, c2 = np.cos(t1), np.cos(t2)

    # 3) Build denominators (add tiny epsilon to avoid zero division)
    eps = 1e-12
    denom_s = n1 * c1 + n2 * c2 + eps
    denom_p = n2 * c1 + n1 * c2 + eps

    # 4) Fresnel formulas for s-polarization
    rs = (n1 * c1 - n2 * c2) / denom_s
    ts = (2 * n1 * c1) / denom_s

    # 5) Fresnel formulas for p-polarization
    rp = (n2 * c1 - n1 * c2) / denom_p
    tp = (2 * n1 * c1) / denom_p

    return rs, rp, ts, tp
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
def Snells_Law(Structure, AOI_degrees):
    """
    Compute light-propagation angles in a multilayer stack via Snell’s Law.

    Parameters:
    - Structure: list of DataFrames, each must have either:
         • 'N' column (complex refractive index), or
         • both 'n' and 'k' columns (real + imaginary parts)
    - AOI_degrees: angle of incidence in degrees (applies to all wavelengths)

    Returns:
    - angles: array, shape (num_layers, num_points)
         Propagation angle in each layer (radians, complex if absorption)
    """
    # 1) Build refractive-index matrix N[layer, point]
    N = np.stack([
        # prefer 'N' if present...
        df["N"].to_numpy()
        # else combine n + i·k if both exist...
        if "N" in df.columns else
        df["n"].to_numpy() + 1j * df["k"].to_numpy()
        # else fill with NaN complex
        if all(col in df.columns for col in ("n", "k")) else
        np.full(df.shape[0], np.nan + 1j * np.nan)
        for df in Structure
    ])

    num_layers, num_points = N.shape

    # 2) Prepare an array to hold angles (layer × wavelength)
    angles = np.zeros((num_layers, num_points), dtype=complex)

    # 3) Convert incidence angle to radians and assign to first layer
    AOI_rad = np.radians(AOI_degrees)
    angles[0, :] = AOI_rad

    # 4) Apply n₁·sinθ₁ = n₂·sinθ₂ → θ₂ = arcsin((n₁/n₂)·sinθ₁) for each interface
    for i in range(1, num_layers):
        ratio = N[i-1] / N[i]
        angles[i, :] = np.arcsin(ratio * np.sin(angles[i-1, :]))

    return angles
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
def Scattering_Matrix(N, angles, d, wavelengths, r, t):
    """
    Compute the 2×2 scattering matrix S for each wavelength in a multilayer stack.

    Parameters:
    - N: array, shape (L, P)
         Complex refractive indices for L layers over P wavelengths.
    - angles: array, shape (L, P)
         Propagation angles in each layer (radians).
    - d: array, shape (L-2,)
         Thicknesses of the film layers (excluding ambient and substrate) in nm.
    - wavelengths: array, shape (P,)
         Wavelengths in nm.
    - r, t: arrays, shape (L-1, P)
         Reflection and transmission coefficients at each interface.

    Returns:
    - S: array, shape (P, 2, 2)
         Total scattering matrix for each wavelength.
    """
    L, P = N.shape

    # 1) Prepare thickness and wavelength for broadcasting:
    d_mat = d[:, None]               # shape (L-2, 1)
    lam = wavelengths[None, :]       # shape (1, P)

    # 2) Build propagation matrices for each film layer:
    #    phase shift = (2π/λ) · n · d · cos(θ)
    phase = (2 * np.pi / lam) * N[1:-1] * d_mat * np.cos(angles[1:-1])
    prop = np.zeros((L-2, P, 2, 2), dtype=complex)
    prop[:, :, 0, 0] = np.exp(-1j * phase)
    prop[:, :, 1, 1] = np.exp( 1j * phase)

    # 3) Build interface matrices for each boundary:
    #    [1/t    r/t]
    #    [r/t    1/t]
    intf = np.zeros((L-1, P, 2, 2), dtype=complex)
    intf[:, :, 0, 0] = 1 / t
    intf[:, :, 0, 1] = r / t
    intf[:, :, 1, 0] = r / t
    intf[:, :, 1, 1] = 1 / t

    # 4) Cascade matrices: start with first interface matrix
    S = intf[0]  # shape (P, 2, 2)

    # 5) For each subsequent layer: propagate then interface
    for i in range(1, L-1):
        S = S @ prop[i-1]   # apply propagation through layer i
        S = S @ intf[i]     # then interface i

    return S
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
def SE_Sim(Structure, AOI, d, f1, f2, f3, f4,
           exp_data=None, fit=False, write_data=False, NCS=False):
    """
    Run spectroscopic ellipsometry simulation on a multilayer stack,
    with optional thickness fitting and Delta adjustment for CompleteEASE.

    Parameters:
    - Structure: list of DataFrames (each with 'N' or 'n','k')
    - AOI:      angle of incidence (degrees)
    - d:        array of film thicknesses (nm), length = len(Structure)-2
    - f1–f4:    delta-offset coefficients
    - exp_data: DataFrame with 'Wavelength (nm)' & 'Delta' for fitting/adjust
    - fit:      if True, optimize d to match exp_data
    - write_data: if True, save results as CSV
    - NCS:      if True, output N/C/S; else Psi/Delta

    Returns:
    - DataFrame (and d_opt if fit=True)
    """

    # --- Helper: simulate Psi & Delta for given d ---
    def run_simulation(d_vals, Structure, AOI, wv):
        # 1) angles in each layer
        angles = Snells_Law(Structure, AOI)
        # 2) Fresnel r/t for s & p
        rs, rp, ts, tp = fresnel_coefficients(N_matrix, angles)
        # 3) build scattering matrices
        Ss = Scattering_Matrix(N_matrix, angles, d_vals, wv, rs, ts)
        Sp = Scattering_Matrix(N_matrix, angles, d_vals, wv, rp, tp)
        # 4) reflection ratios and rho
        Rp = Sp[:,1,0] / Sp[:,0,0]
        Rs = Ss[:,1,0] / Ss[:,0,0]
        rho = np.conj(Rp / Rs)
        # 5) compute Psi, Delta (unwrap phase)
        psi = np.arctan(np.abs(rho)).real
        delta = np.unwrap(np.angle(rho))
        return np.degrees(psi), np.degrees(delta)

    # --- Helper: objective for fitting (MSE of Delta) ---
    def objective_function(d_vals, Structure, AOI, wv, exp_wv, exp_delta):
        _, sim_delta = run_simulation(d_vals, Structure, AOI, wv)
        # interpolate to experimental wavelengths
        interp = interp1d(wv, sim_delta, fill_value='extrapolate')
        diff = exp_delta - np.mod(interp(exp_wv)+180,360)+180
        return np.mean(diff**2)

    # --- Helper: adjust Delta for CompleteEASE convention ---
    def adjust_delta(delta, wv, exp_data):
        # default normalize to [-180,180]
        norm = lambda x: np.mod(x+180,360)-180
        if exp_data is None or 'Delta' not in exp_data:
            return norm(delta)
        exp_wv = exp_data['Wavelength (nm)']; exp_d = exp_data['Delta']
        # find first positive→negative transition
        idx = np.where((exp_d[:-1]>0)&(exp_d[1:]<0))[0]
        delta_adj = norm(delta)
        if idx.size:
            trans = exp_wv.iloc[idx[0]+1]
            # force negative after transition
            delta_adj[wv>=trans] = np.minimum(delta_adj[wv>=trans], delta_adj[wv>=trans]-360)
        return delta_adj

    # 1) extract wavelength axis and build N_matrix
    wv = Structure[0]['Wavelength (nm)'].to_numpy()
    N_matrix = np.stack([
        df['N'].to_numpy() if 'N' in df
        else df['n'].to_numpy()+1j*df['k'].to_numpy()
        for df in Structure
    ])

    # 2) fit thicknesses if requested
    if fit and exp_data is not None:
        exp_wv = exp_data['Wavelength (nm)'].to_numpy()
        exp_d  = exp_data['Delta'].to_numpy()
        res = minimize(objective_function, d,
                       args=(Structure, AOI, wv, exp_wv, exp_d),
                       method='Nelder-Mead',
                       bounds=[(0,None)]*len(d),
                       options={'maxiter':1000})
        d_opt = res.x
        psi, delta = run_simulation(d_opt, Structure, AOI, wv)
    else:
        d_opt = d
        psi, delta = run_simulation(d_opt, Structure, AOI, wv)

    # 3) apply CompleteEASE Delta adjustment
    delta = adjust_delta(delta, wv, exp_data)

    # 4) add optional delta offsets
    factor = 5000/(wv*10)
    delta += factor*(f1 + f2*factor**2 + f3*factor**4 + f4*(1/factor)**2)

    # 5) interpolate outputs to exp_data grid if needed
    if exp_data is not None:
        interp_psi  = interp1d(wv, psi, fill_value='extrapolate')
        interp_delta= interp1d(wv, delta, fill_value='extrapolate')
        psi, delta = interp_psi(exp_wv), interp_delta(exp_wv)
        wv = exp_wv

    # 6) build result DataFrame
    if NCS:
        Nout = np.cos(2*psi)
        Cout = np.sin(2*psi)*np.cos(delta)
        Sout = np.sin(2*psi)*np.sin(delta)
        df = pd.DataFrame({'Wavelength (nm)': wv, 'N':Nout, 'C':Cout, 'S':Sout})
    else:
        df = pd.DataFrame({'Wavelength (nm)': wv, 'Psi':psi, 'Delta':delta})

    # 7) optionally write CSV with naming row
    if write_data:
        label = '_'.join(f"{Structure[i+1].name}_{d_opt[i]:.2f}nm"
                         for i in range(len(d_opt)))
        df.name = label
        header = pd.DataFrame([[label]+['']*(df.shape[1]-1)], columns=df.columns)
        df = pd.concat([header,df], ignore_index=True)
        ts = datetime.now().strftime("%Y%m%d-%H%M%S-%f")[:-3]
        df.to_csv(f"trial_{ts}.csv", index=False)

    # 8) return results
    return (df, d_opt) if fit else df
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
def Bruggeman_EMA_Roussel(Material1, Material2, c):
    """
    Compute effective composite material via Bruggeman EMA (Roussel variant).

    Parameters:
    - Material1: DataFrame (e.g., void), must have 'N' or ('e1','e2')
    - Material2: DataFrame, must have 'N' or ('e1','e2')
    - c: volume fraction of Material2 (0.0–1.0)

    Returns:
    - DataFrame with wavelength, dielectric (e, e1, e2), optical constants (n, k), and N
    """
    # 1) Ensure each material has complex index column 'N'
    for mat in (Material1, Material2):
        if 'N' not in mat.columns:
            if 'e1' in mat.columns and 'e2' in mat.columns:
                mat['N'] = np.sqrt(mat['e1'] + 1j * mat['e2'])
            else:
                raise ValueError("Material must have 'N' or both 'e1' and 'e2'.")

    # 2) Extract arrays
    wv = Material1['Wavelength (nm)'].to_numpy()
    N1, N2 = Material1['N'].to_numpy(), Material2['N'].to_numpy()

    # 3) Bruggeman formula: solve for effective dielectric e
    p = N1 / N2
    b = 0.25 * ((3 * c - 1) * ((1/p) - p) + p)
    z = b + np.sqrt(b**2 + 0.5)
    e = z * N1 * N2

    # 4) Convert dielectric to optical constants n, k
    e1, e2 = e.real, e.imag
    magnitude = np.sqrt(e1**2 + e2**2)
    n = np.sqrt(0.5 * (e1 + magnitude))
    k = np.sqrt(0.5 * (magnitude - e1))
    N_eff = n + 1j * k

    # 5) Build DataFrame
    df = pd.DataFrame({
        'Wavelength (nm)': wv,
        'e': e,
        'e1': e1,
        'e2': e2,
        'n': n,
        'k': k,
        'N': N_eff
    })

    # 6) Annotate name and metadata
    frac1, frac2 = round(1-c, 2), round(c, 2)
    name1 = getattr(Material1, 'name', 'Material1')
    name2 = getattr(Material2, 'name', 'Material2')
    df.name = f"{name1}_{frac1}_{name2}_{frac2}"
    df.attrs['source_info'] = {
        'model': 'EMA (Bruggeman-Roussel)',
        'components': [name1, name2],
        'volume_fraction': {name2: frac2, name1: frac1}
    }

    return df
# -----------------------------------------------------------------------------

# =============================================================================
# Functions from the Cody–Lorentz Workbook & Material Generators
# Equations 1b–31 for Lorentz/Cody–Lorentz oscillators and TL/CL materials
# =============================================================================

def L(E, Amp, Br, Eo):  # Eq. 1b
    """Lorentz broadening term."""
    return (Amp * Br * Eo * E) / ((E**2 - Eo**2)**2 + (Br**2 * E**2))


def Z(Eo, Br):  # Eq. 9
    """Auxiliary root z = √(Eo² – Br²/2)."""
    return (Eo**2 - (Br**2)/2)**0.5


def X(Eo, Br):  # Eq. 10
    """Auxiliary root x = √(4·Eo² – Br²)."""
    return (4*Eo**2 - Br**2)**0.5


def Gt(E, Eg):  # Eq. 4
    """Transition function Gt = (E – Eg)² / E²."""
    return (E - Eg)**2 / E**2


def Ld(E, Eo, Br):  # Eq. 17
    """Denominator for Cot/Dot: (E² – Eo²)² + Br²·E²."""
    return (E**2 - Eo**2)**2 + (Br**2 * E**2)


def Cot(E, Eg, Eo, Br):  # Eq. 11
    """Cot term."""
    return (E * Gt(E, Eg)) / (2 * Ld(E, Eo, Br))


def Dot(E, Eg, Eo, Br):  # Eq. 12
    """Dot term."""
    return - (E + Eg)**2 / (2 * E * Ld(E, Eo, Br))


def A3t(E, Eg, Eo, Br):  # Eq. 13
    """Coefficient a3t = –(Cot + Dot)."""
    return -(Cot(E, Eg, Eo, Br) + Dot(E, Eg, Eo, Br))


def A2t(E, Eg, Eo, Br):  # Eq. 14
    """Coefficient a2t = –E·(Cot – Dot)."""
    return -E * (Cot(E, Eg, Eo, Br) - Dot(E, Eg, Eo, Br))


def A1t(E, Eg, Eo, Br):  # Eq. 15
    """Coefficient a1t."""
    z = Z(Eo, Br)
    return - (E**2 - 2*z**2) * (Cot(E, Eg, Eo, Br) + Dot(E, Eg, Eo, Br))


def Aot(E, Eg, Eo, Br):  # Eq. 16
    """Coefficient a0t."""
    z = Z(Eo, Br)
    return 1 - E*(E**2 - 2*z**2)*(Cot(E, Eg, Eo, Br) - Dot(E, Eg, Eo, Br))


def I1t(Eo, Br, Et):  # Eq. 6
    """Integral I1t."""
    x, z = X(Eo, Br), Z(Eo, Br)
    return (1/(2*x*Br)) * (math.pi - 2*math.atan(2*(Et**2 - z**2)/(x*Br)))


def Ioat(Eo, Br, Et):  # Eq. 7
    """Integral Ioat."""
    x = X(Eo, Br)
    return (1/(2*Br)) * (math.pi - math.atan((2*Et + x)/Br) + math.atan((-2*Et + x)/Br))


def Iobt(Eo, Br, Et):  # Eq. 8
    """Integral Iobt."""
    x = X(Eo, Br)
    return (1/(4*x)) * math.log((Et**2 + Eo**2 + x*Et)/(Et**2 + Eo**2 - x*Et))


def ITL(E, Eg, Eo, Br, A, Et):  # Eqs. 5–8
    """Full Tauc–Lorentz integral."""
    aot = Aot(E, Eg, Eo, Br)
    a1t = A1t(E, Eg, Eo, Br)
    a2t = A2t(E, Eg, Eo, Br)
    a3t = A3t(E, Eg, Eo, Br)
    cot = Cot(E, Eg, Eo, Br)
    dot = Dot(E, Eg, Eo, Br)
    z = Z(Eo, Br)
    ld = Ld(Et, Eo, Br)
    i1t = I1t(Eo, Br, Et)
    ioat = Ioat(Eo, Br, Et)
    iobt = Iobt(Eo, Br, Et)
    coeff = (2*A*Eo*Br) / math.pi
    t1 = a3t * (z**2 * i1t - math.log(ld**0.25))
    t2 = a2t * (ioat + iobt)
    t3 = a1t * i1t
    t4 = aot * ((ioat - iobt) / Eo**2)
    try:
        t5 = cot * math.log(abs(E - Et))
    except ValueError:
        t5 = 0
    t6 = dot * math.log(E + Et)
    return coeff * (t1 + t2 + t3 + t4 - t5 - t6)


def Fsq(Ep, Eg):  # Eq. 29
    """Fsq = Ep² + Eg²."""
    return Ep**2 + Eg**2


def Ksq(Ep, Eg, Eo, Br):  # Eq. 30
    """Ksq term."""
    return 2*Fsq(Ep, Eg) + 2*Z(Eo, Br)**2 - 4*Eg**2


def Ycube(Ep, Eg, Eo, Br):  # Eq. 31
    """Ycube term."""
    return Eo**4 + Fsq(Ep, Eg)*(Ksq(Ep, Eg, Eo, Br) - Fsq(Ep, Eg)) - 4*Eg**2 * Ksq(Ep, Eg, Eo, Br)


def Gc(E, Eg, Ep):  # Eq. 18
    """Cody–Lorentz transition function."""
    return (E - Eg)**2 / ((E - Eg)**2 + Ep**2)


def Ioc(Ep, Eg, Et):  # Eq. 20
    """Integral Ioc."""
    return (1 / Ep) * (math.pi/2 - math.atan((Et - Eg)/Ep))


def Coc(E, Eg, Ep, Eo, Br):  # Eq. 21
    """Term Coc."""
    return E*Gc(E, Eg, Ep) / (2*Ld(E, Eo, Br))


def Doc(E, Eg, Ep, Eo, Br):  # Eq. 22
    """Term Doc."""
    return -E*(E + Eg)**2 / (2*Ld(E, Eo, Br)*((E+Eg)**2 + Ep**2))


def Boc(E, Ep, Eg, Eo, Br):  # Eq. 23
    """Term Boc."""
    y = Ycube(Ep, Eg, Eo, Br)
    fsq = Fsq(Ep, Eg)
    ksq = Ksq(Ep, Eg, Eo, Br)
    ld = Ld(E, Eo, Br)
    coc = Coc(E, Eg, Ep, Eo, Br)
    doc = Doc(E, Eg, Ep, Eo, Br)
    num = y*fsq * (ld*((1/E)*(coc-doc) + 2*Eg*ksq*(coc+doc)/y) - 1)
    den = (ksq - fsq)*fsq*y + Eo**4*y + 4*Eg**2*fsq*ksq**2
    return num / den


def B1c(E, Ep, Eg, Eo, Br):  # Eq. 24
    """Term B1c."""
    return ((2*Eg*Ksq(Ep, Eg, Eo, Br)*Boc(E, Ep, Eg, Eo, Br)
             - Ld(E, Eo, Br)*(Coc(E, Eg, Ep, Eo, Br)+Doc(E, Eg, Ep, Eo, Br)))
            / Ycube(Ep, Eg, Eo, Br))


def A3c(E, Ep, Eg, Eo, Br):  # Eq. 25
    """Coefficient A3c."""
    return -(B1c(E, Ep, Eg, Eo, Br) + Coc(E, Eg, Ep, Eo, Br) + Doc(E, Eg, Ep, Eo, Br))


def A2c(E, Ep, Eg, Eo, Br):  # Eq. 26
    """Coefficient A2c."""
    return -(Boc(E, Ep, Eg, Eo, Br)
             + 2*Eg*B1c(E, Ep, Eg, Eo, Br)
             + E*(Coc(E, Eg, Ep, Eo, Br)-Doc(E, Eg, Ep, Eo, Br)))


def A1c(E, Ep, Eg, Eo, Br):  # Eq. 27
    """Coefficient A1c."""
    z = Z(Eo, Br)
    return -(2*Eg*Boc(E, Ep, Eg, Eo, Br)
             - (Ksq(Ep, Eg, Eo, Br) - Fsq(Ep, Eg))*B1c(E, Ep, Eg, Eo, Br)
             + (E**2 - 2*z**2)*(Coc(E, Eg, Ep, Eo, Br)+Doc(E, Eg, Ep, Eo, Br)))


def Aoc(E, Ep, Eg, Eo, Br):  # Eq. 28
    """Coefficient Aoc."""
    return (1
            + (Ksq(Ep, Eg, Eo, Br) - Fsq(Ep, Eg))*Boc(E, Ep, Eg, Eo, Br)
            + 2*Eg*Ksq(Ep, Eg, Eo, Br)*B1c(E, Ep, Eg, Eo, Br)
            - E*((E**2) - 2*Z(Eo, Br)**2)*(Coc(E, Eg, Ep, Eo, Br)-Doc(E, Eg, Ep, Eo, Br)))


def ITL_C(E, Ep, Eg, Eo, Br, A, Et):  # Eq. 19
    """Combined TL–Cody integral."""
    tl = ITL(E, Eg, Eo, Br, A, Et)
    ioc = Ioc(Ep, Eg, Et)
    boc = Boc(E, Ep, Eg, Eo, Br)
    b1c = B1c(E, Ep, Eg, Eo, Br)
    return tl + (2*A*Eo*Br)/math.pi*(b1c*(Eg*ioc - math.log(((Et-Eg)**2+Ep**2)**0.5)) + boc*ioc)


def ICL(E, Ep, Eg, Eo, Br, A, Et):
    """Final Cody–Lorentz material function."""
    return ITL_C(E, Ep, Eg, Eo, Br, A, Et)

def Get_TL_Material(E, Eg, Eo, Br, Amp, Egt, E_inf, wv):
    """Generate DataFrame of Tauc–Lorentz dielectric function."""
    E = np.array(E)
    e2 = np.zeros_like(E); mask = E > Egt
    e2[mask] = Gt(E[mask], Eg) * L(E[mask], Amp, Br, Eo)
    e1 = np.array([E_inf + ITL(Ei, Eg, Eo, Br, Amp, Egt) for Ei in E])
    e = e1 + 1j*e2
    mag = np.sqrt(e1**2 + e2**2)
    n = np.sqrt((mag + e1)/2); k = np.sqrt((mag - e1)/2)
    df = pd.DataFrame({
        'Wavelength (nm)': wv,
        'Energy (eV)': E, 'e1': e1, 'e2': e2, 'e': e,
        'n': n, 'k': k, 'N': n+1j*k
    })
    df.name = f"TL_Eg{Eg}_Eo{Eo}_Br{Br}_A{Amp}_Et{Egt}_Ei{E_inf}"
    df.attrs['source_info'] = dict(model='TL', Eg=Eg, E0=Eo, Br=Br, A=Amp, Et=Egt, Einf=E_inf)
    return df

def Get_CL_Material(E, Ep, Eg, Eo, Br, Amp, Egt, E_inf, wv):
    """Generate DataFrame of Cody–Lorentz dielectric function."""
    E = np.array(E)
    e2 = np.zeros_like(E); mask = E > Egt
    e2[mask] = Gc(E[mask], Eg, Ep) * L(E[mask], Amp, Br, Eo)
    e1 = np.array([E_inf + ICL(Ei, Ep, Eg, Eo, Br, Amp, Egt) for Ei in E])
    e = e1 + 1j*e2
    mag = np.sqrt(e1**2 + e2**2)
    n = np.sqrt((mag + e1)/2); k = np.sqrt((mag - e1)/2)
    df = pd.DataFrame({
        'Wavelength (nm)': wv,
        'Energy (eV)': E, 'e1': e1, 'e2': e2, 'e': e,
        'n': n, 'k': k, 'N': n+1j*k
    })
    df.name = f"CL_Ep{Ep}_Eg{Eg}_Eo{Eo}_Br{Br}_A{Amp}_Et{Egt}_Ei{E_inf}"
    df.attrs['source_info'] = dict(model='CL', Ep=Ep, Eg=Eg, E0=Eo, Br=Br, A=Amp, Et=Egt, Einf=E_inf)
    return df

# =============================================================================

# -----------------------------------------------------------------------------
def Lorentz(E, An, Br, En, einf=1.0):
    """
    Compute the Lorentz oscillator dielectric function.

    Parameters:
    - E:     array of photon energies (eV)
    - An:    oscillator amplitude
    - Br:    broadening parameter
    - En:    resonance energy
    - einf:  high-frequency permittivity (default 1.0)

    Returns:
    - DataFrame with columns:
        • 'Energy (eV)'  
        • 'e1' (real dielectric)  
        • 'e2' (imaginary dielectric)  
        • 'e'  (complex dielectric)
    """

    # 1) Ensure energies are in an array
    E = np.array(E)

    # 2) Compute complex dielectric: ε = ε_inf + (An·Br·En) / (En² − E² − i·Br·E)
    numerator   = An * Br * En
    denominator = En**2 - E**2 - 1j * Br * E
    dielectric  = einf + numerator / denominator

    # 3) Split into real (e1) and imaginary (e2) parts
    e1 = dielectric.real
    e2 = dielectric.imag

    # 4) Build DataFrame
    df = pd.DataFrame({
        'Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 5) Annotate name and source_info for metadata
    df.name = f"A_{An}_Br_{Br}_En_{En}_Einf_{einf}"
    df.attrs["source_info"] = {
        "model":   "Lorentz",
        "An":      An,
        "Br":      Br,
        "En":      En,
        "Einf":    einf
    }

    return df
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
def drude_epsilon(E, rho_n, tau_fs):
    """
    Compute Drude-model dielectric function for free carriers.

    Parameters:
    - E:       array of photon energies (eV)
    - rho_n:   material resistivity (Ohm·cm)
    - tau_fs:  carrier scattering time (fs)

    Returns:
    - DataFrame with:
        • 'Energy (eV)'  
        • 'e1' (real part of ε)  
        • 'e2' (imaginary part of ε)  
        • 'e'  (complex dielectric function)
    """
    # 1) Physical constants
    hbar     = 6.582119569e-16   # Reduced Planck constant [eV·s]
    eps0     = 8.854e-14         # Vacuum permittivity [F/cm]
    tau      = tau_fs * 1e-15    # Convert scattering time from fs to s

    # 2) Drude numerator & denominator
    #    ε(ω) = -ħ² / [ε₀·ρ_n·(τ·E² + i·ħ·E)]
    numerator   = -hbar**2
    denominator = eps0 * rho_n * (tau * E**2 + 1j * hbar * E)

    # 3) Compute complex dielectric function
    dielectric = numerator / denominator

    # 4) Separate real and imaginary parts
    e1 = dielectric.real
    e2 = dielectric.imag

    # 5) Assemble results into a DataFrame
    df = pd.DataFrame({
        'Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 6) Annotate metadata for traceability
    df.name = f"Drude_rho{rho_n}_tau{tau_fs}"
    df.attrs["source_info"] = {
        "model":   "Drude",
        "rho_n":   rho_n,
        "tau_fs":  tau_fs
    }

    return df
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
def Sellmeier(A_uv, A_ir, En, e_inf, E):
    """
    Compute the real dielectric function using the Sellmeier model.

    Parameters:
    - A_uv:   UV absorption strength coefficient
    - A_ir:   IR absorption strength coefficient
    - En:     resonance energy (eV)
    - e_inf:  high-frequency permittivity
    - E:      array of photon energies (eV)

    Returns:
    - DataFrame with:
        • 'Energy (eV)'  
        • 'e1' (real dielectric)  
        • 'e2' (imaginary part, zero)  
        • 'e'  (complex dielectric = e1 + 0j)
    """
    # 1) Compute components
    UV_dielectric = A_uv / (En**2 - E**2)
    IR_dielectric = -A_ir / (E**2)
    e1 = UV_dielectric + IR_dielectric + e_inf

    # 2) Imaginary part is zero for Sellmeier
    e2 = np.zeros_like(E)
    dielectric = e1 + 0j

    # 3) Build DataFrame
    df = pd.DataFrame({
        'Energy (eV)': E,
        'e1':          e1,
        'e2':          e2,
        'e':           dielectric
    })

    # 4) Add metadata
    df.attrs["source_info"] = {
        "model":  "Sellmeier",
        "A_uv":   A_uv,
        "A_ir":   A_ir,
        "En":     En,
        "e_inf":  e_inf
    }
    df.name = f"Sellmeier_Auv_{A_uv}_Air_{A_ir}_En_{En}_Einf_{e_inf}"

    return df
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
def sumosscilator(df_list):
    """
    Combine multiple oscillator DataFrames into a single composite dielectric.

    Parameters:
    - df_list: list of DataFrames, each with:
        • 'Energy (eV)'
        • 'e1' (real part)
        • 'e2' (imaginary part)

    Returns:
    - DataFrame containing:
        • summed 'e1' and 'e2'
        • complex dielectric 'e = e1 + 1j·e2'
        • preserved 'Energy (eV)' axis
        • metadata listing each component's source_info
    """
    # 1) Use the energy axis from the first oscillator
    E = df_list[0]["Energy (eV)"].values

    # 2) Sum real and imaginary contributions across all oscillators
    e1_total = sum(df["e1"].values for df in df_list)
    e2_total = sum(df["e2"].values for df in df_list)
    e_total  = e1_total + 1j * e2_total

    # 3) Build the composite DataFrame
    df = pd.DataFrame({
        "Energy (eV)": E,
        "e1":          e1_total,
        "e2":          e2_total,
        "e":           e_total
    })

    # 4) Attach metadata listing each component's model info
    df.attrs["source_info"] = {
        "model":      "Composite",
        "components": [
            d.attrs.get("source_info", {"model": "Unknown"})
            for d in df_list
        ]
    }

    return df
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# === Step 1: Load CSVs ===
os.chdir(r"C:\Users\mutha\Downloads\ANN-Spectroscopic-Ellipsometry-main\ANN-Spectroscopic-Ellipsometry-main\Optical_Properties")


SLG       = pd.read_csv("Junda_SLG.csv");      SLG.name      = "SLG"
Void      = pd.read_csv("Void.csv");           Void.name     = "Void"
Si_JAW    = pd.read_csv("Si_JAW.csv");         Si_JAW.name   = "Si_JAW"
NTVE_JAW  = pd.read_csv("NTVE_JAW.csv");       NTVE_JAW.name = "NTVE_JAW"

# === Step 2: Ensure each DataFrame has a complex refractive index 'N' ===
def ensure_N(df):
    """
    Add or compute 'N' = complex refractive index:
      • keep existing 'N'
      • combine 'n' + i·'k'
      • or convert dielectric 'e' → N via sqrt
    """
    if "N" in df.columns:
        return df
    if {"n", "k"}.issubset(df.columns):
        df["N"] = df["n"] + 1j * df["k"]
    elif "e" in df.columns:
        def dielectric_to_index(eps):
            eps1, eps2 = eps.real, eps.imag
            mag = np.sqrt(eps1**2 + eps2**2)
            return np.sqrt((mag + eps1)/2) + 1j*np.sqrt((mag - eps1)/2)
        df["N"] = df["e"].apply(dielectric_to_index)
    else:
        raise ValueError("DataFrame needs 'N', or both 'n','k', or 'e'")
    return df

Void     = ensure_N(Void)
SLG      = ensure_N(SLG)
Si_JAW   = ensure_N(Si_JAW)
NTVE_JAW = ensure_N(NTVE_JAW)

# === Step 3: Set up spectral axis & tag metadata ===
E  = SLG["Energy (eV)"].to_numpy()
wv = SLG["Wavelength (nm)"].to_numpy()

Void.attrs     = {"model": "Void",         "source": "Void.csv"}
NTVE_JAW.attrs = {"model": "NativeOxide",  "source": "NTVE_JAW.csv"}
Si_JAW.attrs   = {"model": "Si_Substrate", "source": "Si_JAW.csv"}

# === Step 4: Helper functions ===
def add_N(df):
    """Compute N = sqrt(eps) when only 'e' is present."""
    df['N'] = np.sqrt(df['e'].to_numpy())

def extract_oscillator_metadata(df):
    """
    Return source_info from df.attrs as a list,
    for JSON compatibility.
    """
    info = df.attrs.get("source_info")
    if isinstance(info, dict):
        return [info]
    elif isinstance(info, list):
        return info
    else:
        return [{"model": "Unknown", "note": "No source_info"}]
# -----------------------------------------------------------------------------
from tqdm import tqdm  # add this alongside your other imports

def generate_simulations(num_files, output_folder, files_per_folder=30000):
    """
    Generate `num_files` random ellipsometry simulations and save:
      • CSV of Psi/Delta results
      • JSON metadata with timestamped filenames

    Files will be partitioned into subfolders named dataset_1, dataset_2, … 
    each containing up to `files_per_folder` examples.

    Progress and ETA are displayed via a progress bar.
    """
    os.makedirs(output_folder, exist_ok=True)

    # --- Constants & budgets ---
    NTVE_thickness      = 1.75
    min_layer_thickness = 0.5
    max_total_thickness = 700.0
    film_budget_max     = max_total_thickness - NTVE_thickness

    # --- Parameter sampling ranges ---
    tl_ranges      = {'Eg': (0.1, 3.0), 'Eo': (0.5, 6.0),
                      'Br': (0.1, 6.0), 'Amp': (0.1, 150.0)}
    sell_ranges    = {'A_uv': (0.0, 50.0), 'A_ir': (0.0, 3.0),
                      'En': (7.0, 8.0), 'e_inf': (0.0, 0.0)}
    drude_ranges   = {'rho_n': (0.0001, 0.009), 'tau_fs': (0.5, 10.0)}
    lorentz_ranges = {'An': (0.01, 1.0), 'Br': (0.05, 0.25), 'En': (0.1, 3.0)}
    AOI_range      = (69.0, 71.0)
    delta_ranges   = {'f1': (-5.0, 5.0), 'f2': (-1.0, 1.0)}

    # --- Helpers ---
    def sample(rng):
        return random.uniform(*rng)

    def round_metadata(obj):
        """Recursively round floats: rho_n→4d, others→3d."""
        if isinstance(obj, dict):
            out = {}
            for k, v in obj.items():
                if isinstance(v, float):
                    out[k] = float(f"{v:.4f}") if k == "rho_n" else float(f"{v:.3f}")
                else:
                    out[k] = round_metadata(v)
            return out
        elif isinstance(obj, list):
            return [round_metadata(v) for v in obj]
        else:
            return obj

    # --- Main loop with progress bar ---
    for i in tqdm(range(1, num_files + 1),
                  desc="Generating sims",
                  unit="sim",
                  ncols=80):
        # determine subfolder bucket
        bucket = (i - 1) // files_per_folder + 1
        subdir = os.path.join(output_folder, f"dataset_{bucket}")
        os.makedirs(subdir, exist_ok=True)

        Eg_global = sample(tl_ranges['Eg'])

        # 1) Bulk layers
        include_SbSeY  = random.choice([True, False])
        include_SbXSeY = random.choice([True, False])
        if not (include_SbSeY or include_SbXSeY):
            include_SbSeY = True

        bulk_layers = []
        def make_bulk(name):
            mats, eos = [], []
            mode, n_tl = random.choice([
                ('TL', 1), ('TL', 2), ('TL', 3), ('TL', 4),
                ('TL+Sell', 1), ('TL+Sell', 2), ('TL+Sell', 3), ('TL+Sell', 4),
            ])
            for _ in range(n_tl):
                while True:
                    Eo = sample(tl_ranges['Eo'])
                    Br = sample(tl_ranges['Br'])
                    if Eo > Eg_global and all(abs(Eo - p) >= 0.5 for p in eos) and Eo**2 > Br**2/2:
                        break
                eos.append(Eo)
                E_inf = 1.0 if len(eos) == 1 else 0.0
                mats.append(Get_TL_Material(E, Eg_global, Eo, Br,
                                           sample(tl_ranges['Amp']),
                                           Eg_global, E_inf, wv))
            if mode == 'TL+Sell':
                mats.append(Sellmeier(
                    sample(sell_ranges['A_uv']),
                    sample(sell_ranges['A_ir']),
                    sample(sell_ranges['En']),
                    sell_ranges['e_inf'][0],
                    E
                ))
            layer = sumosscilator(mats)
            layer.name = name
            add_N(layer)
            return layer

        if include_SbSeY:
            bulk_layers.append(make_bulk("SbSeY"))
        if include_SbXSeY:
            bulk_layers.append(make_bulk("SbXSeY"))

        # 2) Metal layers
        metal_count  = random.choice([0, 1, 2])
        metal_layers = []
        metal_thicks = [sample((0.5, 25.0)) for _ in range(metal_count)]

        if metal_count == 2:
            rho_n = sample(drude_ranges['rho_n'])
            An    = sample(lorentz_ranges['An'])
            Br_l  = sample(lorentz_ranges['Br'])
            En_l  = sample(lorentz_ranges['En'])
            taus  = [sample(drude_ranges['tau_fs']) for _ in range(2)]
            for idx, tau in enumerate(taus, start=1):
                dr = drude_epsilon(E, rho_n, tau)
                lo = Lorentz(E, An, Br_l, En_l)
                m  = sumosscilator([dr, lo])
                m.name = f"Metal{idx}"
                add_N(m)
                metal_layers.append(m)
        else:
            for idx in range(metal_count):
                dr = drude_epsilon(E, sample(drude_ranges['rho_n']),
                                   sample(drude_ranges['tau_fs']))
                lo = Lorentz(E,
                             sample(lorentz_ranges['An']),
                             sample(lorentz_ranges['Br']),
                             sample(lorentz_ranges['En']))
                m  = sumosscilator([dr, lo])
                m.name = f"Metal{idx+1}"
                add_N(m)
                metal_layers.append(m)

        # 3) Optional surface roughness
        sr_layer = None
        sr_thk   = None
        if random.random() < 0.5:
            base = bulk_layers[0] if bulk_layers else NTVE_JAW
            sr_layer = Bruggeman_EMA_Roussel(Void, base, 0.5)
            sr_layer.name = "SurfaceRoughness"
            add_N(sr_layer)
            sr_thk = sample((0.5, 25.0))

        film_layers = ([sr_layer] if sr_layer is not None else []) + bulk_layers + metal_layers

        # 4) Thickness allocation
        n_bulk    = len(bulk_layers)
        remaining = film_budget_max - (sr_thk or 0) - sum(metal_thicks)
        if n_bulk > 0:
            min_tot = n_bulk * min_layer_thickness
            if remaining > min_tot:
                extra = random.uniform(0, remaining - min_tot)
                weights = np.random.rand(n_bulk)
                weights /= weights.sum()
                bulk_thicks = list(weights * extra + min_layer_thickness)
            else:
                bulk_thicks = [min_layer_thickness] * n_bulk
        else:
            bulk_thicks = []

        parts = ([sr_thk] if sr_thk else []) + bulk_thicks + metal_thicks + [NTVE_thickness]
        d = np.round(np.array(parts), 3)

        # 5) Run simulation
        Structure = [Void] + film_layers + [NTVE_JAW, Si_JAW]
        AOI       = sample(AOI_range)
        f1, f2    = sample(delta_ranges['f1']), sample(delta_ranges['f2'])
        df = SE_Sim(Structure, AOI, d, f1, f2, 0.0, 0.0,
                    write_data=False, NCS=False)

        # 6) Save CSV & JSON
        now = datetime.now()
        ts  = now.strftime("%Y%m%d-%H%M%S-%f")[:-3]
        base = f"sim_{i:06d}_{ts}"
        df.to_csv(os.path.join(subdir, f"{base}.csv"), index=False)

        metadata = {
            "timestamp":       now.isoformat(),
            "AOI":             AOI,
            "delta_offsets":   {"f1": f1, "f2": f2, "f3": 0.0, "f4": 0.0},
            "thicknesses (nm)": d.tolist(),
            "layers":          [lay.name for lay in Structure],
            "oscillators":     {}
        }
        skip = {"Void", "SurfaceRoughness", "NTVE_JAW", "Si_JAW"}
        for lay in film_layers:
            if lay.name not in skip:
                metadata["oscillators"][lay.name] = extract_oscillator_metadata(lay)
        metadata = round_metadata(metadata)
        with open(os.path.join(subdir, f"{base}.json"), "w") as jf:
            json.dump(metadata, jf, indent=2)

    print("✅ All simulations generated.")
# -----------------------------------------------------------------------------


# ---------------------Plugin your output file location------------------------

generate_simulations(
    num_files=210,
    output_folder=r"C:\Users\mutha\OneDrive - University of Toledo\Desktop\ElDataset",
    files_per_folder=30
)

# -----------------------------------------------------------------------------





    